In [12]:
#$env:PYTHONPATH="src"      # PowerShell
import sys
import os

project_root = os.path.abspath("../src")
sys.path.append(project_root)

%load_ext autoreload
%autoreload
from curves.yield_curve import YieldCurve
from products.coupon_bond import CouponBond
from products.future import EquityFuture
from products.swap import InterestRateSwap
from products.option import EuropeanCall
from models.black_scholes import BlackScholesModel
from models.calibration import ImpliedVolatilitySolver




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
curve = YieldCurve(
    maturities=[1, 2, 3, 5],
    zero_rates=[0.02, 0.025, 0.03, 0.035]
)

print(curve.zero_rate(4))   # interpolation entre 3y et 5y
print(curve.discount_factor(4))


0.0325
0.8780954309205613


In [5]:
bond = CouponBond(nominal=100, coupon_rate=0.05, maturity=5, payment_frequency=1)

print("Bond price =", bond.price(curve))

Bond price = 106.76026075076305


In [6]:
fut = EquityFuture(
    spot=4000,       # S&P 500 spot
    rate=0.03,       # 3%
    dividend_yield=0.015,  # 1.5%
    maturity=0.5     # 6 months
)

print("Future price =", fut.price())


Future price = 4030.1127817781353


In [7]:
swap = InterestRateSwap(
    notional=1_000_000,
    fixed_rate=0.03,
    payment_times=[1, 2, 3]
)

print("Fixed Leg PV =", swap.pv_fixed_leg(curve))
print("Floating Leg PV =", swap.pv_floating_leg(curve))
print("Swap Value =", swap.price(curve))
print("Swap rate =", swap.swap_rate(curve))

Fixed Leg PV = 85360.77849236093
Floating Leg PV = 86068.81472877182
Swap Value = 708.0362364108878
Swap rate = 0.02328967326889742


In [11]:
call = EuropeanCall(strike=100, maturity=1.0)
model = BlackScholesModel(spot=105, rate=0.03)

vol = 0.2

print("Call price =", model.call_price(call, vol))
print("Delta =", model.call_delta(call, vol))
print("Gamma =", model.gamma(call, vol))
print("Vega =", model.vega(call, vol))


Call price = 12.638755916496294
Delta = 0.6893295440636719
Gamma = 0.01681549194942603
Vega = 37.078159748484396


In [13]:
# Market price of the call
market_price = 12.50

# Solver
solver = ImpliedVolatilitySolver(model)

iv = solver.implied_vol_call(call, market_price)

print("Implied volatility =", iv)

Implied volatility = 0.19625254622411945
